In [1]:
import json
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime as dt
import time

In [17]:
def get_theme_list():
    r = requests.get("https://api.poets.org/api/taxonomy_term/themes")
    themes_raw = r.json()['data']
    return [{'id': t['attributes']['drupal_internal__tid'], 'name': t['attributes']['name']} for t in themes_raw]

In [20]:
themes = get_theme_list()

In [22]:
r = requests.get("https://api.poets.org/api/poems?field_poem_themes_target_id=851&page=1")

In [55]:
def get_json_poems(json):
    poems = json['rows']
    return pd.DataFrame(poems)

def get_theme_poems(id):
    df = pd.DataFrame()
    r = requests.get("https://api.poets.org/api/poems?field_poem_themes_target_id={}&page=1".format(id))
    data = r.json()
    df = df.append(get_json_poems(data))
    df['page'] = 1
    max_page = data['pager']['total_pages']
    for i in range(2, max_page):
        r = requests.get("https://api.poets.org/api/poems?field_poem_themes_target_id={}&page={}".format(id, i))
        new = get_json_poems(r.json())
        new['page'] = i
        df = df.append(new)
    df = df.reset_index().drop(columns=['index'])
    df['theme_id'] = id
    return df

In [64]:
def get_all_themes(themes):
    df = pd.DataFrame()
    for t in themes:
        new_df = get_theme_poems(t['id'])
        df = df.append(new_df)
        print(t)
    return df

In [65]:
poems = get_all_themes(themes)

{'id': 851, 'name': 'Afterlife'}
{'id': 856, 'name': 'Aging'}
{'id': 861, 'name': 'Ambition'}
{'id': 866, 'name': 'America'}
{'id': 871, 'name': 'American Revolution'}
{'id': 876, 'name': 'Ancestry'}
{'id': 881, 'name': 'Anger'}
{'id': 886, 'name': 'Animals'}
{'id': 891, 'name': 'Apocalypse'}
{'id': 896, 'name': 'Audio'}
{'id': 901, 'name': 'Beauty'}
{'id': 906, 'name': 'Beginning'}
{'id': 911, 'name': 'Birds'}
{'id': 916, 'name': 'Body'}
{'id': 921, 'name': 'Brothers'}
{'id': 926, 'name': 'Carpe Diem'}
{'id': 931, 'name': 'Cats'}


/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


{'id': 936, 'name': 'Childhood'}
{'id': 941, 'name': 'Cities'}
{'id': 946, 'name': 'Clothing'}
{'id': 951, 'name': 'Cooking'}
{'id': 956, 'name': 'Creation'}
{'id': 961, 'name': 'Dance'}
{'id': 966, 'name': 'Daughters'}
{'id': 971, 'name': 'Death'}
{'id': 976, 'name': 'Desire'}
{'id': 981, 'name': 'Divorce'}
{'id': 986, 'name': 'Dogs'}
{'id': 991, 'name': 'Dreams'}
{'id': 996, 'name': 'Drinking'}
{'id': 1001, 'name': 'Drugs'}
{'id': 1006, 'name': 'Earth'}
{'id': 1011, 'name': 'Eating'}
{'id': 1016, 'name': 'Enemies'}
{'id': 1021, 'name': 'Fathers'}
{'id': 1026, 'name': 'Flight'}
{'id': 1031, 'name': 'Flowers'}
{'id': 1036, 'name': 'For Mom'}
{'id': 1041, 'name': 'For Teens'}
{'id': 1046, 'name': 'Friendship'}
{'id': 1051, 'name': 'Future'}
{'id': 1056, 'name': 'Gardens'}
{'id': 1061, 'name': 'Gender'}
{'id': 1066, 'name': 'Ghosts'}
{'id': 1071, 'name': 'Gratitude'}
{'id': 1076, 'name': 'Grief'}
{'id': 1081, 'name': 'Happiness'}
{'id': 1086, 'name': 'Heartache'}
{'id': 1091, 'name': 'He

In [81]:
poems.to_json('theme_poems.json', orient='records')